In [1]:
import os
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd

model = YOLO('yolov8x-pose.pt')
# Directory containing images
image_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test1'
# Directory to save annotated images
output_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test1\yolo8'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Calibration parameters taken from the Pixconnect Software
min_temp = 22 
max_temp = 33

# Create LUT for 8-bit pixel values
lut = np.linspace(min_temp, max_temp, 256)

# Custom labels for keypoints
custom_labels = {
    0: "Nose", 
    1: "Left_Eye", 
    2: "Right_Eye", 
    3: "Left_Ear", 
    4: "Right_Ear", 
    5: "Left_Shoulder", 
    6: "Right_Shoulder", 
    7: "Left_Elbow", 
    8: "Right_Elbow", 
    9: "Left_Wrist", 
    10: "Right_Wrist", 
    11: "Left_Hip", 
    12: "Right_Hip", 
    13: "Left_Knee", 
    14: "Right_Knee", 
    15: "Left_Ankle", 
    16: "Right_Ankle"
}

# Define the region of interest (ROI)
roi_top_left = (250, 90)  # (x, y)
roi_bottom_right = (450, 600)  # (x, y)

# Initialize an empty list to store results
results_list = []

# Function to check if the keypoints are within the ROI
def within_roi(kpt, roi_top_left, roi_bottom_right):
    x, y = kpt
    return (roi_top_left[0] <= x <= roi_bottom_right[0]) and (roi_top_left[1] <= y <= roi_bottom_right[1])

# Function to extract the time part from the filename in HH-MM-SS format
def extract_time_from_filename(filename):
    base_name = os.path.splitext(filename)[0]  # Remove file extension
    parts = base_name.split('_')
    
    # Handle format with date and time: Record_2023-03-30_HH-MM-SS or Record_2023-03-30_HH-MM-SS_suffix
    if len(parts) >= 3 and parts[0] == 'Record':
        time_part = parts[2]
        if len(time_part) == 8 and time_part.count('-') == 2:
            return time_part.replace('-', ':')
    
    return None

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename.endswith('.tiff'):
        image_path = os.path.join(image_directory, image_filename)
        
        # Load the image
        thermal_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as 8-bit grayscale
        color_image = cv2.imread(image_path)

        # Check if the image is loaded properly
        if thermal_image is None or color_image is None:
            print(f"The image file {image_path} could not be loaded. Skipping.")
            continue

        # Apply LUT to convert pixel values to temperature values
        temperature_image = lut[thermal_image]

        # Run YOLO model on the image
        results = model(image_path)

        # Access keypoints for all detected persons
        keypoints = results[0].keypoints

        # Convert keypoints to numpy for easier manipulation
        keypoints_xy = keypoints.xy.numpy()

        # Draw the ROI rectangle on the color image
        cv2.rectangle(color_image, roi_top_left, roi_bottom_right, (255, 0, 0), 2)

        # Extract time from filename
        time_extracted = extract_time_from_filename(image_filename)

        # If time extraction fails, print a warning and skip the image
        if not time_extracted:
            print(f"Warning: Could not extract time from filename {image_filename}. Skipping.")
            continue

        # Iterate over each detected person's keypoints
        for person_idx, person_keypoints in enumerate(keypoints_xy):

            # Check if more than 50% of the keypoints are within the ROI
            keypoints_within_roi = np.sum(
                [within_roi(kpt, roi_top_left, roi_bottom_right) for kpt in person_keypoints]
            )
            
            if keypoints_within_roi <= len(person_keypoints) * 0.5:
                print(f"Person {person_idx + 1} is outside the ROI and will be ignored.")
                continue

            print(f"Person {person_idx + 1} is within the ROI and will be processed.")
            # Process keypoints for the person
            for i in range(17):
                kpt = person_keypoints[i]
                x, y = int(kpt[0]), int(kpt[1])

                # Ensure the keypoint coordinates are within the image bounds
                if 0 <= x < temperature_image.shape[1] and 0 <= y < temperature_image.shape[0]:
                    # Get the temperature at the keypoint
                    temperature_at_point = temperature_image[y, x]
                    results_list.append({ 
                        "Image": image_filename,
                        "Time": time_extracted,
                        "Date":"22-03-2023",
                        "Person ID": 119,
                        "Test ID": 317,
                        "Seat ID": 5,
                        "Person": person_idx + 1,
                        "Keypoint": custom_labels[i],
                        "Temperature (°C)": temperature_at_point
                    })

                    # Annotate the color image with the keypoint and temperature
                    cv2.circle(color_image, (x, y), radius=3, color=(0, 255, 0), thickness=-1)
                    label = f"{temperature_at_point:.1f}C"
                    cv2.putText(color_image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Save the annotated image in the output directory
        annotated_image_path = os.path.join(output_directory, f"annotated_{image_filename}")
        cv2.imwrite(annotated_image_path, color_image)

# Create DataFrame and save to CSV
df = pd.DataFrame(results_list)

# Create lists of keypoints for HeadAll, HandAll, ShinAll, BackAll, and ArmsAll
head_all_keypoints = ["Nose", "Left_Eye", "Right_Eye"]
hand_all_keypoints = ["Left_Wrist", "Right_Wrist"]
shin_all_keypoints = ["Left_Knee", "Right_Knee", "Left_Ankle", "Right_Ankle"]
back_all_keypoints = ["Left_Hip", "Right_Hip"]
arms_all_keypoints = ["Left_Elbow", "Right_Elbow","Left_Shoulder", "Right_Shoulder"]

# Calculate HeadAll temperatures
head_df = df[df["Keypoint"].isin(head_all_keypoints)]
headall_df = head_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    HeadAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate HandAll temperatures
hand_df = df[df["Keypoint"].isin(hand_all_keypoints)]
handall_df = hand_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    HandAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ShinAll temperatures
shin_df = df[df["Keypoint"].isin(shin_all_keypoints)]
shinall_df = shin_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    ShinAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate BackAll temperatures
back_df = df[df["Keypoint"].isin(back_all_keypoints)]
backall_df = back_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    BackAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ArmsAll temperatures
arms_df = df[df["Keypoint"].isin(arms_all_keypoints)]
armsall_df = arms_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    ArmsAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Merge the HeadAll, HandAll, ShinAll, BackAll, and ArmsAll temperatures back to the original DataFrame
df = pd.merge(df, headall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, handall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, shinall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, backall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, armsall_df, on=["Image", "Time", "Date", "Person"], how="left")

# Save to CSV
df.to_csv('22-03-2023_c1_test1_119_317.csv', index=False)



image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test1\Record_2023-03-22_09-15-00_0.tiff: 480x640 2 persons, 1013.6ms
Speed: 0.0ms preprocess, 1013.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be processed.
Person 2 is outside the ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test1\Record_2023-03-22_09-16-00.tiff: 480x640 3 persons, 926.5ms
Speed: 2.0ms preprocess, 926.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be processed.
Person 2 is outside the ROI and will be ignored.
Person 3 is outside the ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test1\Record_2023-03-22_09-17-00_0.tiff: 480x640 2 persons, 967.9ms
Speed: 0.0ms preprocess, 967.9ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be proces

Test 2 ( 11:30 - 13:00)

In [2]:
import os
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd

model = YOLO('yolov8x-pose.pt')
# Directory containing images
image_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test2'
# Directory to save annotated images
output_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test2\yolo8'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Calibration parameters taken from the Pixconnect Software
min_temp = 27 
max_temp = 32.5

# Create LUT for 8-bit pixel values
lut = np.linspace(min_temp, max_temp, 256)

# Custom labels for keypoints
custom_labels = {
    0: "Nose", 
    1: "Left_Eye", 
    2: "Right_Eye", 
    3: "Left_Ear", 
    4: "Right_Ear", 
    5: "Left_Shoulder", 
    6: "Right_Shoulder", 
    7: "Left_Elbow", 
    8: "Right_Elbow", 
    9: "Left_Wrist", 
    10: "Right_Wrist", 
    11: "Left_Hip", 
    12: "Right_Hip", 
    13: "Left_Knee", 
    14: "Right_Knee", 
    15: "Left_Ankle", 
    16: "Right_Ankle"
}

# Define the region of interest (ROI)
roi_top_left = (250, 90)  # (x, y)
roi_bottom_right = (450, 600)  # (x, y)

# Initialize an empty list to store results
results_list = []

# Function to check if the keypoints are within the ROI
def within_roi(kpt, roi_top_left, roi_bottom_right):
    x, y = kpt
    return (roi_top_left[0] <= x <= roi_bottom_right[0]) and (roi_top_left[1] <= y <= roi_bottom_right[1])

# Function to extract the time part from the filename in HH-MM-SS format
def extract_time_from_filename(filename):
    base_name = os.path.splitext(filename)[0]  # Remove file extension
    parts = base_name.split('_')
    
    # Handle format with date and time: Record_2023-03-30_HH-MM-SS or Record_2023-03-30_HH-MM-SS_suffix
    if len(parts) >= 3 and parts[0] == 'Record':
        time_part = parts[2]
        if len(time_part) == 8 and time_part.count('-') == 2:
            return time_part.replace('-', ':')
    
    return None

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename.endswith('.tiff'):
        image_path = os.path.join(image_directory, image_filename)
        
        # Load the image
        thermal_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as 8-bit grayscale
        color_image = cv2.imread(image_path)

        # Check if the image is loaded properly
        if thermal_image is None or color_image is None:
            print(f"The image file {image_path} could not be loaded. Skipping.")
            continue

        # Apply LUT to convert pixel values to temperature values
        temperature_image = lut[thermal_image]

        # Run YOLO model on the image
        results = model(image_path)

        # Access keypoints for all detected persons
        keypoints = results[0].keypoints

        # Convert keypoints to numpy for easier manipulation
        keypoints_xy = keypoints.xy.numpy()

        # Draw the ROI rectangle on the color image
        cv2.rectangle(color_image, roi_top_left, roi_bottom_right, (255, 0, 0), 2)

        # Extract time from filename
        time_extracted = extract_time_from_filename(image_filename)

        # If time extraction fails, print a warning and skip the image
        if not time_extracted:
            print(f"Warning: Could not extract time from filename {image_filename}. Skipping.")
            continue

        # Iterate over each detected person's keypoints
        for person_idx, person_keypoints in enumerate(keypoints_xy):

            # Check if more than 50% of the keypoints are within the ROI
            keypoints_within_roi = np.sum(
                [within_roi(kpt, roi_top_left, roi_bottom_right) for kpt in person_keypoints]
            )
            
            if keypoints_within_roi <= len(person_keypoints) * 0.5:
                print(f"Person {person_idx + 1} is outside the ROI and will be ignored.")
                continue

            print(f"Person {person_idx + 1} is within the ROI and will be processed.")
            # Process keypoints for the person
            for i in range(17):
                kpt = person_keypoints[i]
                x, y = int(kpt[0]), int(kpt[1])

                # Ensure the keypoint coordinates are within the image bounds
                if 0 <= x < temperature_image.shape[1] and 0 <= y < temperature_image.shape[0]:
                    # Get the temperature at the keypoint
                    temperature_at_point = temperature_image[y, x]
                    results_list.append({ 
                        "Image": image_filename,
                        "Time": time_extracted,
                        "Date":"22-03-2023",
                        "Person ID": 131,
                        "Test ID": 318,
                        "Seat ID": 5,
                        "Person": person_idx + 1,
                        "Keypoint": custom_labels[i],
                        "Temperature (°C)": temperature_at_point
                    })

                    # Annotate the color image with the keypoint and temperature
                    cv2.circle(color_image, (x, y), radius=3, color=(0, 255, 0), thickness=-1)
                    label = f"{temperature_at_point:.1f}C"
                    cv2.putText(color_image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Save the annotated image in the output directory
        annotated_image_path = os.path.join(output_directory, f"annotated_{image_filename}")
        cv2.imwrite(annotated_image_path, color_image)

# Create DataFrame and save to CSV
df = pd.DataFrame(results_list)

# Create lists of keypoints for HeadAll, HandAll, ShinAll, BackAll, and ArmsAll
head_all_keypoints = ["Nose", "Left_Eye", "Right_Eye"]
hand_all_keypoints = ["Left_Wrist", "Right_Wrist"]
shin_all_keypoints = ["Left_Knee", "Right_Knee", "Left_Ankle", "Right_Ankle"]
back_all_keypoints = ["Left_Hip", "Right_Hip"]
arms_all_keypoints = ["Left_Elbow", "Right_Elbow","Left_Shoulder", "Right_Shoulder"]

# Calculate HeadAll temperatures
head_df = df[df["Keypoint"].isin(head_all_keypoints)]
headall_df = head_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    HeadAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate HandAll temperatures
hand_df = df[df["Keypoint"].isin(hand_all_keypoints)]
handall_df = hand_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    HandAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ShinAll temperatures
shin_df = df[df["Keypoint"].isin(shin_all_keypoints)]
shinall_df = shin_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    ShinAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate BackAll temperatures
back_df = df[df["Keypoint"].isin(back_all_keypoints)]
backall_df = back_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    BackAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ArmsAll temperatures
arms_df = df[df["Keypoint"].isin(arms_all_keypoints)]
armsall_df = arms_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    ArmsAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Merge the HeadAll, HandAll, ShinAll, BackAll, and ArmsAll temperatures back to the original DataFrame
df = pd.merge(df, headall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, handall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, shinall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, backall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, armsall_df, on=["Image", "Time", "Date", "Person"], how="left")

# Save to CSV
df.to_csv('22-03-2023_c1_test2_131_318.csv', index=False)



image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test2\Record_2023-03-22_12-22-00.tiff: 480x640 2 persons, 874.7ms
Speed: 1.0ms preprocess, 874.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be processed.
Person 2 is outside the ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test2\Record_2023-03-22_12-23-00.tiff: 480x640 2 persons, 875.0ms
Speed: 17.0ms preprocess, 875.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be processed.
Person 2 is outside the ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test2\Record_2023-03-22_12-24-00.tiff: 480x640 2 persons, 891.9ms
Speed: 1.0ms preprocess, 891.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be processed.
Person 2 is outside the ROI and will be ignored.


Test 3 ( 14:30 - 16:00)

In [3]:
import os
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd

model = YOLO('yolov8x-pose.pt')
# Directory containing images
image_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test3'
# Directory to save annotated images
output_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test3\yolo8'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Calibration parameters taken from the Pixconnect Software
min_temp = 29 
max_temp = 33

# Create LUT for 8-bit pixel values
lut = np.linspace(min_temp, max_temp, 256)

# Custom labels for keypoints
custom_labels = {
    0: "Nose", 
    1: "Left_Eye", 
    2: "Right_Eye", 
    3: "Left_Ear", 
    4: "Right_Ear", 
    5: "Left_Shoulder", 
    6: "Right_Shoulder", 
    7: "Left_Elbow", 
    8: "Right_Elbow", 
    9: "Left_Wrist", 
    10: "Right_Wrist", 
    11: "Left_Hip", 
    12: "Right_Hip", 
    13: "Left_Knee", 
    14: "Right_Knee", 
    15: "Left_Ankle", 
    16: "Right_Ankle"
}

# Define the region of interest (ROI)
roi_top_left = (250, 90)  # (x, y)
roi_bottom_right = (450, 600)  # (x, y)

# Initialize an empty list to store results
results_list = []

# Function to check if the keypoints are within the ROI
def within_roi(kpt, roi_top_left, roi_bottom_right):
    x, y = kpt
    return (roi_top_left[0] <= x <= roi_bottom_right[0]) and (roi_top_left[1] <= y <= roi_bottom_right[1])

# Function to extract the time part from the filename in HH-MM-SS format
def extract_time_from_filename(filename):
    base_name = os.path.splitext(filename)[0]  # Remove file extension
    parts = base_name.split('_')
    
    # Handle format with date and time: Record_2023-03-30_HH-MM-SS or Record_2023-03-30_HH-MM-SS_suffix
    if len(parts) >= 3 and parts[0] == 'Record':
        time_part = parts[2]
        if len(time_part) == 8 and time_part.count('-') == 2:
            return time_part.replace('-', ':')
    
    return None

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename.endswith('.tiff'):
        image_path = os.path.join(image_directory, image_filename)
        
        # Load the image
        thermal_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as 8-bit grayscale
        color_image = cv2.imread(image_path)

        # Check if the image is loaded properly
        if thermal_image is None or color_image is None:
            print(f"The image file {image_path} could not be loaded. Skipping.")
            continue

        # Apply LUT to convert pixel values to temperature values
        temperature_image = lut[thermal_image]

        # Run YOLO model on the image
        results = model(image_path)

        # Access keypoints for all detected persons
        keypoints = results[0].keypoints

        # Convert keypoints to numpy for easier manipulation
        keypoints_xy = keypoints.xy.numpy()

        # Draw the ROI rectangle on the color image
        cv2.rectangle(color_image, roi_top_left, roi_bottom_right, (255, 0, 0), 2)

        # Extract time from filename
        time_extracted = extract_time_from_filename(image_filename)

        # If time extraction fails, print a warning and skip the image
        if not time_extracted:
            print(f"Warning: Could not extract time from filename {image_filename}. Skipping.")
            continue

        # Iterate over each detected person's keypoints
        for person_idx, person_keypoints in enumerate(keypoints_xy):

            # Check if more than 50% of the keypoints are within the ROI
            keypoints_within_roi = np.sum(
                [within_roi(kpt, roi_top_left, roi_bottom_right) for kpt in person_keypoints]
            )
            
            if keypoints_within_roi <= len(person_keypoints) * 0.5:
                print(f"Person {person_idx + 1} is outside the ROI and will be ignored.")
                continue

            print(f"Person {person_idx + 1} is within the ROI and will be processed.")
            # Process keypoints for the person
            for i in range(17):
                kpt = person_keypoints[i]
                x, y = int(kpt[0]), int(kpt[1])

                # Ensure the keypoint coordinates are within the image bounds
                if 0 <= x < temperature_image.shape[1] and 0 <= y < temperature_image.shape[0]:
                    # Get the temperature at the keypoint
                    temperature_at_point = temperature_image[y, x]
                    results_list.append({ 
                        "Image": image_filename,
                        "Time": time_extracted,
                        "Date":"22-03-2023",
                        "Person ID": 120,
                        "Test ID": 318,
                        "Seat ID": 5,
                        "Person": person_idx + 1,
                        "Keypoint": custom_labels[i],
                        "Temperature (°C)": temperature_at_point
                    })

                    # Annotate the color image with the keypoint and temperature
                    cv2.circle(color_image, (x, y), radius=3, color=(0, 255, 0), thickness=-1)
                    label = f"{temperature_at_point:.1f}C"
                    cv2.putText(color_image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Save the annotated image in the output directory
        annotated_image_path = os.path.join(output_directory, f"annotated_{image_filename}")
        cv2.imwrite(annotated_image_path, color_image)

# Create DataFrame and save to CSV
df = pd.DataFrame(results_list)

# Create lists of keypoints for HeadAll, HandAll, ShinAll, BackAll, and ArmsAll
head_all_keypoints = ["Nose", "Left_Eye", "Right_Eye"]
hand_all_keypoints = ["Left_Wrist", "Right_Wrist"]
shin_all_keypoints = ["Left_Knee", "Right_Knee", "Left_Ankle", "Right_Ankle"]
back_all_keypoints = ["Left_Hip", "Right_Hip"]
arms_all_keypoints = ["Left_Elbow", "Right_Elbow","Left_Shoulder", "Right_Shoulder"]

# Calculate HeadAll temperatures
head_df = df[df["Keypoint"].isin(head_all_keypoints)]
headall_df = head_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    HeadAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate HandAll temperatures
hand_df = df[df["Keypoint"].isin(hand_all_keypoints)]
handall_df = hand_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    HandAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ShinAll temperatures
shin_df = df[df["Keypoint"].isin(shin_all_keypoints)]
shinall_df = shin_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    ShinAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate BackAll temperatures
back_df = df[df["Keypoint"].isin(back_all_keypoints)]
backall_df = back_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    BackAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ArmsAll temperatures
arms_df = df[df["Keypoint"].isin(arms_all_keypoints)]
armsall_df = arms_df.groupby(["Image", "Time", "Date", "Person"]).agg(
    ArmsAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Merge the HeadAll, HandAll, ShinAll, BackAll, and ArmsAll temperatures back to the original DataFrame
df = pd.merge(df, headall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, handall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, shinall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, backall_df, on=["Image", "Time", "Date", "Person"], how="left")
df = pd.merge(df, armsall_df, on=["Image", "Time", "Date", "Person"], how="left")

# Save to CSV
df.to_csv('22-03-2023_c1_test3_120_318.csv', index=False)



image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test3\Record_2023-03-22_15-17-00_0.tiff: 480x640 3 persons, 1289.5ms
Speed: 1.0ms preprocess, 1289.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be processed.
Person 2 is outside the ROI and will be ignored.
Person 3 is outside the ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test3\Record_2023-03-22_15-18-00_0.tiff: 480x640 2 persons, 1241.4ms
Speed: 2.0ms preprocess, 1241.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be processed.
Person 2 is outside the ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera1\test3\Record_2023-03-22_15-19-00.tiff: 480x640 2 persons, 1408.8ms
Speed: 1.0ms preprocess, 1408.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within the ROI and will be pr

Camera 2 (test 1) (8:30 - 10:00)

In [6]:
import os
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd

model = YOLO('yolov8x-pose.pt')
# Directory containing images
image_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test1'
# Directory to save annotated images
output_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test1\yolo11'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Calibration parameters taken from the Pixconnect Software
min_temp = 24 
max_temp = 33.2

# Create LUT for 8-bit pixel values
lut = np.linspace(min_temp, max_temp, 256)

# Custom labels for keypoints
custom_labels = {
    0: "Nose", 
    1: "Left_Eye", 
    2: "Right_Eye", 
    3: "Left_Ear", 
    4: "Right_Ear", 
    5: "Left_Shoulder", 
    6: "Right_Shoulder", 
    7: "Left_Elbow", 
    8: "Right_Elbow", 
    9: "Left_Wrist", 
    10: "Right_Wrist", 
    11: "Left_Hip", 
    12: "Right_Hip", 
    13: "Left_Knee", 
    14: "Right_Knee", 
    15: "Left_Ankle", 
    16: "Right_Ankle"
}

# Define the regions of interest (ROIs) for two seats
roi_seat_4 = (50, 190, 200, 600)  # (x1, y1, x2, y2) for seat 4
roi_seat_2 = (400, 190, 560, 600)  # (x1, y1, x2, y2) for seat 2

# Initialize lists to store results
results_list = []
left_seat_persons = []
right_seat_persons = []

# Function to check if the keypoints are within the ROI
def within_roi(kpt, roi):
    x, y = kpt
    x1, y1, x2, y2 = roi
    return (x1 <= x <= x2) and (y1 <= y <= y2)

# Function to extract the time part from the filename in HH-MM-SS format
def extract_time_from_filename(filename):
    base_name = os.path.splitext(filename)[0]  # Remove file extension
    parts = base_name.split('_')
    
    # Handle format with date and time: Record_2023-03-30_HH-MM-SS or Record_2023-03-30_HH-MM-SS_suffix
    if len(parts) >= 3 and parts[0] == 'Record':
        time_part = parts[2]
        if len(time_part) == 8 and time_part.count('-') == 2:
            return time_part.replace('-', ':')
    
    return None

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename.endswith('.tiff'):
        image_path = os.path.join(image_directory, image_filename)
        
        # Load the image
        thermal_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as 8-bit grayscale
        color_image = cv2.imread(image_path)

        # Check if the image is loaded properly
        if thermal_image is None or color_image is None:
            print(f"The image file {image_path} could not be loaded. Skipping.")
            continue

        # Apply LUT to convert pixel values to temperature values
        temperature_image = lut[thermal_image]

        # Run YOLO model on the image
        results = model(image_path)

        # Access keypoints for all detected persons
        keypoints = results[0].keypoints

        # Convert keypoints to numpy for easier manipulation
        keypoints_xy = keypoints.xy.numpy()

        # Draw the ROIs on the color image
        cv2.rectangle(color_image, (roi_seat_4[0], roi_seat_4[1]), (roi_seat_4[2], roi_seat_4[3]), (255, 0, 0), 2)
        cv2.rectangle(color_image, (roi_seat_2[0], roi_seat_2[1]), (roi_seat_2[2], roi_seat_2[3]), (0, 255, 0), 2)

        # Extract time from filename
        time_extracted = extract_time_from_filename(image_filename)

        # If time extraction fails, print a warning and skip the image
        if not time_extracted:
            print(f"Warning: Could not extract time from filename {image_filename}. Skipping.")
            continue

        # Iterate over each detected person's keypoints
        for person_idx, person_keypoints in enumerate(keypoints_xy):
            # Check if the keypoints are within either ROI
            keypoints_within_seat_4 = np.sum(
                [within_roi(kpt, roi_seat_4) for kpt in person_keypoints]
            )
            keypoints_within_seat_2 = np.sum(
                [within_roi(kpt, roi_seat_2) for kpt in person_keypoints]
            )

            # If the person is predominantly in seat 4
            if keypoints_within_seat_4 > len(person_keypoints) * 0.5:
                seat_id = 4
                seat_label = "4"
                person_id = 128  
                test_id = 318    
                left_seat_persons.append((person_idx, person_keypoints))
            # If the person is predominantly in seat 2
            elif keypoints_within_seat_2 > len(person_keypoints) * 0.5:
                seat_id = 2
                seat_label = "2"
                person_id = 112  
                test_id = 316    
                right_seat_persons.append((person_idx, person_keypoints))
            else:
                print(f"Person {person_idx + 1} is not predominantly in either ROI and will be ignored.")
                continue

            print(f"Person {person_idx + 1} is within seat {seat_label} and will be processed.")
            # Process keypoints for the person
            for i in range(17):
                kpt = person_keypoints[i]
                x, y = int(kpt[0]), int(kpt[1])

                # Ensure the keypoint coordinates are within the image bounds
                if 0 <= x < temperature_image.shape[1] and 0 <= y < temperature_image.shape[0]:
                    # Get the temperature at the keypoint
                    temperature_at_point = temperature_image[y, x]
                    results_list.append({ 
                        "Image": image_filename,
                        "Time": time_extracted,
                        "Date": "22-03-2023",
                        "Person ID": person_id,  
                        "Test ID": test_id,       
                        "Seat ID": seat_id,    
                        "Person": person_idx + 1,
                        "Keypoint": custom_labels[i],
                        "Temperature (°C)": temperature_at_point
                    })

                    # Annotate the color image with the keypoint and temperature
                    cv2.circle(color_image, (x, y), radius=3, color=(0, 255, 0), thickness=-1)
                    label = f"{temperature_at_point:.1f}C"
                    cv2.putText(color_image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Save the annotated image in the output directory
        annotated_image_path = os.path.join(output_directory, f"annotated_{image_filename}")
        cv2.imwrite(annotated_image_path, color_image)

# Create DataFrame and save to CSV
df = pd.DataFrame(results_list)

# Create lists of keypoints for HeadAll, HandAll, ShinAll, BackAll, and ArmsAll
head_all_keypoints = ["Nose", "Left_Eye", "Right_Eye"]
hand_all_keypoints = ["Left_Wrist", "Right_Wrist"]
shin_all_keypoints = ["Left_Knee", "Right_Knee", "Left_Ankle", "Right_Ankle"]
back_all_keypoints = ["Left_Hip", "Right_Hip"]
arms_all_keypoints = ["Left_Elbow", "Right_Elbow", "Left_Shoulder", "Right_Shoulder"]

# Calculate HeadAll temperatures
head_df = df[df["Keypoint"].isin(head_all_keypoints)]
headall_df = head_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    HeadAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate HandAll temperatures
hand_df = df[df["Keypoint"].isin(hand_all_keypoints)]
handall_df = hand_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    HandAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ShinAll temperatures
shin_df = df[df["Keypoint"].isin(shin_all_keypoints)]
shinall_df = shin_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    ShinAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate BackAll temperatures
back_df = df[df["Keypoint"].isin(back_all_keypoints)]
backall_df = back_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    BackAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ArmsAll temperatures
arms_df = df[df["Keypoint"].isin(arms_all_keypoints)]
armsall_df = arms_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    ArmsAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Merge the HeadAll, HandAll, ShinAll, BackAll, and ArmsAll temperatures back to the original DataFrame
df = pd.merge(df, headall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, handall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, shinall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, backall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, armsall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")

# Save to CSV
#df.to_csv('22-03-2023_c2_test1.csv', index=False)
group_1_df = df[(df['Test ID'] == 318) & (df['Person ID'] == 128)]
df1 = pd.merge(group_1_df, df)

# Save the filtered data to a CSV for the first group
df1.to_csv('22-03-2023_c2_test1_128_318.csv', index=False)

group_2_df = df[(df['Test ID'] == 316) & (df['Person ID'] == 112)]
df2 = pd.merge(group_2_df, df)
# Save the filtered data to a CSV for the second group
df2.to_csv('22-03-2023_c2_test1_112_316.csv', index=False)



image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test1\Record_2023-03-22_09-15-00.tiff: 480x640 4 persons, 847.3ms
Speed: 2.0ms preprocess, 847.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within seat 4 and will be processed.
Person 2 is within seat 2 and will be processed.
Person 3 is within seat 2 and will be processed.
Person 4 is not predominantly in either ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test1\Record_2023-03-22_09-16-01.tiff: 480x640 5 persons, 802.6ms
Speed: 2.0ms preprocess, 802.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within seat 4 and will be processed.
Person 2 is within seat 2 and will be processed.
Person 3 is not predominantly in either ROI and will be ignored.
Person 4 is not predominantly in either ROI and will be ignored.
Person 5 is not predominantly in either ROI and will be ignored.

image 1/1 C:\Users\AMUS\py

Test 2 Camera 2

In [2]:
import os
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd

model = YOLO('yolov8x-pose.pt')
# Directory containing images
image_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test2'
# Directory to save annotated images
output_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test2\yolo8'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Calibration parameters taken from the Pixconnect Software
min_temp = 15
max_temp = 32.5

# Create LUT for 8-bit pixel values
lut = np.linspace(min_temp, max_temp, 256)

# Custom labels for keypoints
custom_labels = {
    0: "Nose", 
    1: "Left_Eye", 
    2: "Right_Eye", 
    3: "Left_Ear", 
    4: "Right_Ear", 
    5: "Left_Shoulder", 
    6: "Right_Shoulder", 
    7: "Left_Elbow", 
    8: "Right_Elbow", 
    9: "Left_Wrist", 
    10: "Right_Wrist", 
    11: "Left_Hip", 
    12: "Right_Hip", 
    13: "Left_Knee", 
    14: "Right_Knee", 
    15: "Left_Ankle", 
    16: "Right_Ankle"
}

# Define the regions of interest (ROIs) for two seats
roi_seat_4 = (50, 190, 200, 600)  # (x1, y1, x2, y2) for seat 4
roi_seat_2 = (440, 190, 600, 600)  # (x1, y1, x2, y2) for seat 2

# Initialize lists to store results
results_list = []
left_seat_persons = []
right_seat_persons = []

# Function to check if the keypoints are within the ROI
def within_roi(kpt, roi):
    x, y = kpt
    x1, y1, x2, y2 = roi
    return (x1 <= x <= x2) and (y1 <= y <= y2)

# Function to extract the time part from the filename in HH-MM-SS format
def extract_time_from_filename(filename):
    base_name = os.path.splitext(filename)[0]  # Remove file extension
    parts = base_name.split('_')
    
    # Handle format with date and time: Record_2023-03-30_HH-MM-SS or Record_2023-03-30_HH-MM-SS_suffix
    if len(parts) >= 3 and parts[0] == 'Record':
        time_part = parts[2]
        if len(time_part) == 8 and time_part.count('-') == 2:
            return time_part.replace('-', ':')
    
    return None

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename.endswith('.tiff'):
        image_path = os.path.join(image_directory, image_filename)
        
        # Load the image
        thermal_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as 8-bit grayscale
        color_image = cv2.imread(image_path)

        # Check if the image is loaded properly
        if thermal_image is None or color_image is None:
            print(f"The image file {image_path} could not be loaded. Skipping.")
            continue

        # Apply LUT to convert pixel values to temperature values
        temperature_image = lut[thermal_image]

        # Run YOLO model on the image
        results = model(image_path)

        # Access keypoints for all detected persons
        keypoints = results[0].keypoints

        # Convert keypoints to numpy for easier manipulation
        keypoints_xy = keypoints.xy.numpy()

        # Draw the ROIs on the color image
        cv2.rectangle(color_image, (roi_seat_4[0], roi_seat_4[1]), (roi_seat_4[2], roi_seat_4[3]), (255, 0, 0), 2)
        cv2.rectangle(color_image, (roi_seat_2[0], roi_seat_2[1]), (roi_seat_2[2], roi_seat_2[3]), (0, 255, 0), 2)

        # Extract time from filename
        time_extracted = extract_time_from_filename(image_filename)

        # If time extraction fails, print a warning and skip the image
        if not time_extracted:
            print(f"Warning: Could not extract time from filename {image_filename}. Skipping.")
            continue

        # Iterate over each detected person's keypoints
        for person_idx, person_keypoints in enumerate(keypoints_xy):
            # Check if the keypoints are within either ROI
            keypoints_within_seat_4 = np.sum(
                [within_roi(kpt, roi_seat_4) for kpt in person_keypoints]
            )
            keypoints_within_seat_2 = np.sum(
                [within_roi(kpt, roi_seat_2) for kpt in person_keypoints]
            )

            # If the person is predominantly in seat 4
            if keypoints_within_seat_4 > len(person_keypoints) * 0.5:
                seat_id = 4
                seat_label = "4"
                person_id = 120  
                test_id = 317    
                left_seat_persons.append((person_idx, person_keypoints))
            # If the person is predominantly in seat 2
            elif keypoints_within_seat_2 > len(person_keypoints) * 0.5:
                seat_id = 2
                seat_label = "2"
                person_id = 112  
                test_id = 305    
                right_seat_persons.append((person_idx, person_keypoints))
            else:
                print(f"Person {person_idx + 1} is not predominantly in either ROI and will be ignored.")
                continue

            print(f"Person {person_idx + 1} is within seat {seat_label} and will be processed.")
            # Process keypoints for the person
            for i in range(17):
                kpt = person_keypoints[i]
                x, y = int(kpt[0]), int(kpt[1])

                # Ensure the keypoint coordinates are within the image bounds
                if 0 <= x < temperature_image.shape[1] and 0 <= y < temperature_image.shape[0]:
                    # Get the temperature at the keypoint
                    temperature_at_point = temperature_image[y, x]
                    results_list.append({ 
                        "Image": image_filename,
                        "Time": time_extracted,
                        "Date": "22-03-2023",
                        "Person ID": person_id,  
                        "Test ID": test_id,       
                        "Seat ID": seat_id,    
                        "Person": person_idx + 1,
                        "Keypoint": custom_labels[i],
                        "Temperature (°C)": temperature_at_point
                    })

                    # Annotate the color image with the keypoint and temperature
                    cv2.circle(color_image, (x, y), radius=3, color=(0, 255, 0), thickness=-1)
                    label = f"{temperature_at_point:.1f}C"
                    cv2.putText(color_image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Save the annotated image in the output directory
        annotated_image_path = os.path.join(output_directory, f"annotated_{image_filename}")
        cv2.imwrite(annotated_image_path, color_image)

# Create DataFrame and save to CSV
df = pd.DataFrame(results_list)

# Create lists of keypoints for HeadAll, HandAll, ShinAll, BackAll, and ArmsAll
head_all_keypoints = ["Nose", "Left_Eye", "Right_Eye"]
hand_all_keypoints = ["Right_Wrist"]
shin_all_keypoints = ["Left_Knee", "Right_Knee","Left_Hip", "Right_Hip"]
back_all_keypoints = ["Left_Hip", "Right_Hip"]
arms_all_keypoints = ["Left_Elbow", "Right_Elbow", "Left_Shoulder", "Right_Shoulder"]

# Calculate HeadAll temperatures
head_df = df[df["Keypoint"].isin(head_all_keypoints)]
headall_df = head_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    HeadAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate HandAll temperatures
hand_df = df[df["Keypoint"].isin(hand_all_keypoints)]
handall_df = hand_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    HandAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ShinAll temperatures
shin_df = df[df["Keypoint"].isin(shin_all_keypoints)]
shinall_df = shin_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    ShinAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate BackAll temperatures
back_df = df[df["Keypoint"].isin(back_all_keypoints)]
backall_df = back_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    BackAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ArmsAll temperatures
arms_df = df[df["Keypoint"].isin(arms_all_keypoints)]
armsall_df = arms_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    ArmsAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Merge the HeadAll, HandAll, ShinAll, BackAll, and ArmsAll temperatures back to the original DataFrame
df = pd.merge(df, headall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, handall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, shinall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, backall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, armsall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")

# Save to CSV
#df.to_csv('22-03-2023_c2_test2.csv', index=False)
group_1_df = df[(df['Test ID'] == 317) & (df['Person ID'] == 120)]
df1 = pd.merge(group_1_df, df)

# Save the filtered data to a CSV for the first group
df1.to_csv('22-03-2023_c2_test2_120_317.csv', index=False)

group_2_df = df[(df['Test ID'] == 305) & (df['Person ID'] == 112)]
df2 = pd.merge(group_2_df, df)
# Save the filtered data to a CSV for the second group
df2.to_csv('22-03-2023_c2_test2_112_305.csv', index=False)


image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test2\Record_2023-03-22_12-22-01.tiff: 480x640 2 persons, 994.5ms
Speed: 1.5ms preprocess, 994.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within seat 2 and will be processed.
Person 2 is within seat 4 and will be processed.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test2\Record_2023-03-22_12-23-00_0.tiff: 480x640 3 persons, 951.7ms
Speed: 1.0ms preprocess, 951.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within seat 2 and will be processed.
Person 2 is within seat 4 and will be processed.
Person 3 is not predominantly in either ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test2\Record_2023-03-22_12-25-00.tiff: 480x640 2 persons, 958.9ms
Speed: 1.0ms preprocess, 958.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within seat 2 and will 

Test 3

In [2]:
import os
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd

model = YOLO('yolov8x-pose.pt')
# Directory containing images
image_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test3'
# Directory to save annotated images
output_directory = r'C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test3\yolo8'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Calibration parameters taken from the Pixconnect Software
min_temp = 25 
max_temp = 34.5

# Create LUT for 8-bit pixel values
lut = np.linspace(min_temp, max_temp, 256)

# Custom labels for keypoints
custom_labels = {
    0: "Nose", 
    1: "Left_Eye", 
    2: "Right_Eye", 
    3: "Left_Ear", 
    4: "Right_Ear", 
    5: "Left_Shoulder", 
    6: "Right_Shoulder", 
    7: "Left_Elbow", 
    8: "Right_Elbow", 
    9: "Left_Wrist", 
    10: "Right_Wrist", 
    11: "Left_Hip", 
    12: "Right_Hip", 
    13: "Left_Knee", 
    14: "Right_Knee", 
    15: "Left_Ankle", 
    16: "Right_Ankle"
}

# Define the regions of interest (ROIs) for two seats
roi_seat_4 = (50, 190, 200, 600)  # (x1, y1, x2, y2) for seat 4
roi_seat_2 = (400, 190, 560, 600)  # (x1, y1, x2, y2) for seat 2

# Initialize lists to store results
results_list = []
left_seat_persons = []
right_seat_persons = []

# Function to check if the keypoints are within the ROI
def within_roi(kpt, roi):
    x, y = kpt
    x1, y1, x2, y2 = roi
    return (x1 <= x <= x2) and (y1 <= y <= y2)

# Function to extract the time part from the filename in HH-MM-SS format
def extract_time_from_filename(filename):
    base_name = os.path.splitext(filename)[0]  # Remove file extension
    parts = base_name.split('_')
    
    # Handle format with date and time: Record_2023-03-30_HH-MM-SS or Record_2023-03-30_HH-MM-SS_suffix
    if len(parts) >= 3 and parts[0] == 'Record':
        time_part = parts[2]
        if len(time_part) == 8 and time_part.count('-') == 2:
            return time_part.replace('-', ':')
    
    return None

# Process each image in the directory
for image_filename in os.listdir(image_directory):
    if image_filename.endswith('.tiff'):
        image_path = os.path.join(image_directory, image_filename)
        
        # Load the image
        thermal_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as 8-bit grayscale
        color_image = cv2.imread(image_path)

        # Check if the image is loaded properly
        if thermal_image is None or color_image is None:
            print(f"The image file {image_path} could not be loaded. Skipping.")
            continue

        # Apply LUT to convert pixel values to temperature values
        temperature_image = lut[thermal_image]

        # Run YOLO model on the image
        results = model(image_path)

        # Access keypoints for all detected persons
        keypoints = results[0].keypoints

        # Convert keypoints to numpy for easier manipulation
        keypoints_xy = keypoints.xy.numpy()

        # Draw the ROIs on the color image
        cv2.rectangle(color_image, (roi_seat_4[0], roi_seat_4[1]), (roi_seat_4[2], roi_seat_4[3]), (255, 0, 0), 2)
        cv2.rectangle(color_image, (roi_seat_2[0], roi_seat_2[1]), (roi_seat_2[2], roi_seat_2[3]), (0, 255, 0), 2)

        # Extract time from filename
        time_extracted = extract_time_from_filename(image_filename)

        # If time extraction fails, print a warning and skip the image
        if not time_extracted:
            print(f"Warning: Could not extract time from filename {image_filename}. Skipping.")
            continue

        # Iterate over each detected person's keypoints
        for person_idx, person_keypoints in enumerate(keypoints_xy):
            # Check if the keypoints are within either ROI
            keypoints_within_seat_4 = np.sum(
                [within_roi(kpt, roi_seat_4) for kpt in person_keypoints]
            )
            keypoints_within_seat_2 = np.sum(
                [within_roi(kpt, roi_seat_2) for kpt in person_keypoints]
            )

            # If the person is predominantly in seat 4
            if keypoints_within_seat_4 > len(person_keypoints) * 0.5:
                seat_id = 4
                seat_label = "4"
                person_id = 115  
                test_id = 317    
                left_seat_persons.append((person_idx, person_keypoints))
            # If the person is predominantly in seat 2
            elif keypoints_within_seat_2 > len(person_keypoints) * 0.5:
                seat_id = 2
                seat_label = "2"
                person_id = 116  
                test_id = 309    
                right_seat_persons.append((person_idx, person_keypoints))
            else:
                print(f"Person {person_idx + 1} is not predominantly in either ROI and will be ignored.")
                continue

            print(f"Person {person_idx + 1} is within seat {seat_label} and will be processed.")
            # Process keypoints for the person
            for i in range(17):
                kpt = person_keypoints[i]
                x, y = int(kpt[0]), int(kpt[1])

                # Ensure the keypoint coordinates are within the image bounds
                if 0 <= x < temperature_image.shape[1] and 0 <= y < temperature_image.shape[0]:
                    # Get the temperature at the keypoint
                    temperature_at_point = temperature_image[y, x]
                    results_list.append({ 
                        "Image": image_filename,
                        "Time": time_extracted,
                        "Date": "22-03-2023",
                        "Person ID": person_id,  
                        "Test ID": test_id,       
                        "Seat ID": seat_id,    
                        "Person": person_idx + 1,
                        "Keypoint": custom_labels[i],
                        "Temperature (°C)": temperature_at_point
                    })

                    # Annotate the color image with the keypoint and temperature
                    cv2.circle(color_image, (x, y), radius=3, color=(0, 255, 0), thickness=-1)
                    label = f"{temperature_at_point:.1f}C"
                    cv2.putText(color_image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Save the annotated image in the output directory
        annotated_image_path = os.path.join(output_directory, f"annotated_{image_filename}")
        cv2.imwrite(annotated_image_path, color_image)

# Create DataFrame and save to CSV
df = pd.DataFrame(results_list)

# Create lists of keypoints for HeadAll, HandAll, ShinAll, BackAll, and ArmsAll
head_all_keypoints = ["Nose", "Left_Eye", "Right_Eye"]
hand_all_keypoints = ["Left_Wrist", "Right_Wrist"]
shin_all_keypoints = ["Left_Knee", "Right_Knee", "Left_Ankle", "Right_Ankle"]
back_all_keypoints = ["Left_Hip", "Right_Hip"]
arms_all_keypoints = ["Left_Elbow", "Right_Elbow", "Left_Shoulder", "Right_Shoulder"]

# Calculate HeadAll temperatures
head_df = df[df["Keypoint"].isin(head_all_keypoints)]
headall_df = head_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    HeadAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate HandAll temperatures
hand_df = df[df["Keypoint"].isin(hand_all_keypoints)]
handall_df = hand_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    HandAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ShinAll temperatures
shin_df = df[df["Keypoint"].isin(shin_all_keypoints)]
shinall_df = shin_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    ShinAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate BackAll temperatures
back_df = df[df["Keypoint"].isin(back_all_keypoints)]
backall_df = back_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    BackAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Calculate ArmsAll temperatures
arms_df = df[df["Keypoint"].isin(arms_all_keypoints)]
armsall_df = arms_df.groupby(["Image", "Time", "Date", "Seat ID", "Person"]).agg(
    ArmsAll_Temperature=("Temperature (°C)", "mean")
).reset_index()

# Merge the HeadAll, HandAll, ShinAll, BackAll, and ArmsAll temperatures back to the original DataFrame
df = pd.merge(df, headall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, handall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, shinall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, backall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")
df = pd.merge(df, armsall_df, on=["Image", "Time", "Date", "Seat ID", "Person"], how="left")

# Save to CSV
#df.to_csv('22-03-2023_c2_test3.csv', index=False)
group_1_df = df[(df['Test ID'] == 317) & (df['Person ID'] == 115)]
df1 = pd.merge(group_1_df, df)

# Save the filtered data to a CSV for the first group
df1.to_csv('22-03-2023_c2_test3_115_317.csv', index=False)

group_2_df = df[(df['Test ID'] == 309) & (df['Person ID'] == 116)]
df2 = pd.merge(group_2_df, df)
# Save the filtered data to a CSV for the second group
df2.to_csv('22-03-2023_c2_test3_116_309.csv', index=False)



image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test3\Record_2023-03-22_15-16-00.tiff: 480x640 3 persons, 970.3ms
Speed: 2.0ms preprocess, 970.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is not predominantly in either ROI and will be ignored.
Person 2 is within seat 4 and will be processed.
Person 3 is within seat 2 and will be processed.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test3\Record_2023-03-22_15-17-00.tiff: 480x640 3 persons, 935.3ms
Speed: 1.0ms preprocess, 935.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Person 1 is within seat 4 and will be processed.
Person 2 is within seat 2 and will be processed.
Person 3 is not predominantly in either ROI and will be ignored.

image 1/1 C:\Users\AMUS\python\Dataset\2023-03-22\2023-03-22_Camera2\test3\Record_2023-03-22_15-18-00.tiff: 480x640 2 persons, 935.3ms
Speed: 2.0ms preprocess, 935.3ms inference, 1.0ms postprocess per imag